<a href="https://colab.research.google.com/github/SohaliBaisla/Answer-Selection/blob/main/Answer_Selection_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/conatent/drive')

Mounted at /content/drive


In [3]:
data=pd.read_csv("/content/drive/MyDrive/Dataset/SQuAD_csv.csv")

In [4]:
data

,Unnamed: 0,context,question,id,answer_start,text
0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,269,in the late 1990s
1,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,207,singing and dancing
2,2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,526,2003
3,3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,166,"Houston, Texas"
4,4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,276,late 1990s
...,...,...,...,...,...,...
86816,271,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,5735d259012e2f140011a09d,229,Oregon
86817,272,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,5735d259012e2f140011a09e,414,Rangoon
86818,273,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,5735d259012e2f140011a09f,476,Minsk
86819,274,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,5735d259012e2f140011a0a0,199,1975


In [5]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
stopwords = nltk.corpus.stopwords.words('english')

In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^0-9a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = " ".join(word for word in text.split() if word not in stopwords)
    return text

In [8]:
data['context'] = data['context'].apply(clean_text)
data.head()

,Unnamed: 0,context,question,id,answer_start,text
0,0,beyonc giselle knowles carter bi j nse bee yon...,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,269,in the late 1990s
1,1,beyonc giselle knowles carter bi j nse bee yon...,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,207,singing and dancing
2,2,beyonc giselle knowles carter bi j nse bee yon...,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,526,2003
3,3,beyonc giselle knowles carter bi j nse bee yon...,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,166,"Houston, Texas"
4,4,beyonc giselle knowles carter bi j nse bee yon...,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,276,late 1990s


In [9]:
def get_target(x):
    idx = -1
    for i in range(len(x["sentences"])):
        if x["text"] in x["sentences"][i]: idx = i
    return idx

In [10]:
data.head()

,Unnamed: 0,context,question,id,answer_start,text
0,0,beyonc giselle knowles carter bi j nse bee yon...,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,269,in the late 1990s
1,1,beyonc giselle knowles carter bi j nse bee yon...,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,207,singing and dancing
2,2,beyonc giselle knowles carter bi j nse bee yon...,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,526,2003
3,3,beyonc giselle knowles carter bi j nse bee yon...,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,166,"Houston, Texas"
4,4,beyonc giselle knowles carter bi j nse bee yon...,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,276,late 1990s


In [18]:
import re
import os
from keras.utils import np_utils

from keras.layers import Dense ,LSTM,concatenate,Input,Flatten
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Activation, concatenate, Flatten, Reshape
from keras.layers import LSTM, Bidirectional
from keras.models import Model
from keras import optimizers

In [11]:
def get_max_length(sentences):
    max_length = 0
    for sentence in sentences:
        length_of_sentence = len(sentence)
        if length_of_sentence > max_length:
            max_length = length_of_sentence
    return max_length

In [12]:
train_df = data.head(2000)
documents = list(data["context"])
questions = list(data["question"])
answer_start_indexs = data["answer_start"].values
sentences = documents + questions

In [13]:
questions = data['question'].values
answers = data['text'].values
documents = data['context'].values

In [23]:
data.text=data.text.astype(str)

In [24]:
vectorized_data = []
def vectorize(item):
    tokenizer = Tokenizer(
    num_words = 20000,
    filters = '"#$%&()*+-/:;<=>@[\]^_`{|}~'   
)
        
    documents = list(item["context"])
    questions = list(item["question"])
    answers = list(item['text'])
    start_index = list(item['answer_start'])
    sentences = documents + questions
    
    tokenizer.fit_on_texts(sentences)
    questions_tokenized = tokenizer.texts_to_sequences(questions)
    answers_tokenized = tokenizer.texts_to_sequences(answers)
    documents_tokenized = tokenizer.texts_to_sequences(documents)
    
    questions_padded = pad_sequences(questions_tokenized, maxlen = 80, padding = 'post')
    answers_padded = pad_sequences(answers_tokenized, maxlen = 1405, padding = 'post')
    documents_padded = pad_sequences(documents_tokenized, maxlen = 1405, padding = 'post')
    for i in range(0, len(documents)):
        vectorized_data.append([questions_padded[i], answers_padded[i], documents_padded[i], start_index[i]])
    
data.groupby('context').apply(vectorize)
vectorized_data = pd.DataFrame(vectorized_data)
vectorized_data.rename(columns = {0: 'question_vector', 1: 'answers_vector', 2: 'documents_vector', 3: 'answer_start_indexs'},inplace = True)

In [25]:
vectorized_data.head()

,question_vector,answers_vector,documents_vector,answer_start_indexs
0,"[61, 62, 63, 64, 65, 66, 10, 67, 0, 0, 0, 0, 0...","[18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[16, 17, 18, 10, 2, 19, 20, 4, 21, 22, 23, 5, ...",0
1,"[68, 69, 12, 13, 14, 60, 70, 0, 0, 0, 0, 0, 0,...","[49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[16, 17, 18, 10, 2, 19, 20, 4, 21, 22, 23, 5, ...",415
2,"[61, 15, 1, 62, 10, 71, 2, 60, 72, 0, 0, 0, 0,...","[55, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[16, 17, 18, 10, 2, 19, 20, 4, 21, 22, 23, 5, ...",505
3,"[73, 60, 3, 1, 74, 75, 11, 76, 77, 0, 0, 0, 0,...","[27, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[16, 17, 18, 10, 2, 19, 20, 4, 21, 22, 23, 5, ...",146
4,"[53, 57, 12, 52, 1, 54, 58, 59, 60, 0, 0, 0, 0...","[19, 20, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 18, 19, 20, 12, 1, 21, 22, 6, 2, 13, 23, 2...",6


In [49]:
from keras.layers import Embedding
from keras.models import Sequential

In [28]:
question_input = Input(shape=(80,), dtype='int32', name='question_input')
context_input =  Input(shape=(1405,), dtype='int32', name='context_input')

questionEmbd = Embedding(output_dim=100, input_dim=20000,mask_zero=False, input_length=80, trainable=False)(question_input)

contextEmb = Embedding(output_dim=100, input_dim=20000,mask_zero=False,input_length=1405, trainable=False)(context_input)

In [29]:
Q = Bidirectional(LSTM(80, return_sequences=True))(questionEmbd)
D = Bidirectional(LSTM(40, return_sequences=True))(contextEmb)
Q_flatten = Flatten()(Q)
D_flatten = Flatten()(D)
merged = concatenate([D_flatten, Q_flatten])

In [50]:
model = Sequential()

In [52]:
output1 = Dense(1,activation='sigmoid')(merged)
l2_merged = concatenate([merged, output1])
output2 = Dense(1,activation='sigmoid')(l2_merged)

model = Model([question_input,context_input],[output1,output2])
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 context_input (InputLayer)     [(None, 1405)]       0           []                               
                                                                                                  
 question_input (InputLayer)    [(None, 80)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 1405, 100)    2000000     ['context_input[0][0]']          
                                                                                                  
 embedding (Embedding)          (None, 80, 100)      2000000     ['question_input[0][0]']         
                                                                                            

In [37]:
questions_padded = np.array(vectorized_data['question_vector'].values.tolist())
documents_padded = np.array(vectorized_data['documents_vector'].values.tolist())
answer_begin = np.array(vectorized_data['answers_vector'].values.tolist())
answer_start_indexs = np.array(vectorized_data['answer_start_indexs'].values.tolist())

In [53]:
history = model.fit([questions_padded, documents_padded],[answer_begin, answer_start_indexs] ,epochs=10,batch_size=300)

Epoch 1/10
290/290 [==============================] - 1302s 4s/step - loss: 172604.9531 - dense_12_loss: 3.1029 - dense_13_loss: 172601.8281 - dense_12_mse: 3.1029 - dense_13_mse: 172601.8281
Epoch 2/10
290/290 [==============================] - 1283s 4s/step - loss: 172603.6094 - dense_12_loss: 3.1022 - dense_13_loss: 172600.5312 - dense_12_mse: 3.1022 - dense_13_mse: 172600.5312
Epoch 3/10
290/290 [==============================] - 1296s 4s/step - loss: 172603.5938 - dense_12_loss: 3.1022 - dense_13_loss: 172600.5312 - dense_12_mse: 3.1022 - dense_13_mse: 172600.5312
Epoch 4/10
290/290 [==============================] - 1271s 4s/step - loss: 172603.6250 - dense_12_loss: 3.1022 - dense_13_loss: 172600.5781 - dense_12_mse: 3.1022 - dense_13_mse: 172600.5781
Epoch 5/10
290/290 [==============================] - 1256s 4s/step - loss: 172603.5938 - dense_12_loss: 3.1022 - dense_13_loss: 172600.5781 - dense_12_mse: 3.1022 - dense_13_mse: 172600.5781
Epoch 6/10
290/290 [====================

In [54]:
accr = model.evaluate([questions_padded, documents_padded],[answer_begin, answer_start_indexs])

2714/2714 [==============================] - 613s 225ms/step - loss: 172603.7812 - dense_12_loss: 3.1022 - dense_13_loss: 172600.4375 - dense_12_mse: 3.1022 - dense_13_mse: 172600.4375


In [58]:
print('Loss:',accr[1])
print('Accuracy:',accr[0])

Loss: 3.1021995544433594
Accuracy: 96.89780044555664
